In [1]:
import boto3
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [2]:
s3 = boto3.client("s3")
all_objects = s3.list_objects(Bucket='dse-cohort5-group5')
all_objects

{'ResponseMetadata': {'RequestId': 'C2FED643128B8591',
  'HostId': '3QECxeVtbzJKaedy3S3MByN0Yt81RzEHvCJYncyLlPeU47W/3KLG+e9E6PIg2Fjq//49zl4ZcRc=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '3QECxeVtbzJKaedy3S3MByN0Yt81RzEHvCJYncyLlPeU47W/3KLG+e9E6PIg2Fjq//49zl4ZcRc=',
   'x-amz-request-id': 'C2FED643128B8591',
   'date': 'Sat, 14 Mar 2020 18:29:35 GMT',
   'x-amz-bucket-region': 'us-west-1',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 1},
 'IsTruncated': False,
 'Marker': '',
 'Contents': [{'Key': 'test/',
   'LastModified': datetime.datetime(2020, 2, 14, 2, 26, 50, tzinfo=tzutc()),
   'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'Size': 0,
   'StorageClass': 'STANDARD',
   'Owner': {'DisplayName': 'kcoakley+cohort5group5',
    'ID': 'fe3a1755292b139ad5397ba537aa0aff32ef6ebba07fe45dcbe486d81e1ccc11'}},
  {'Key': 'wildfire_capstone/gridMet.parquet.gz',
   'LastModified': datetime.datetime(2020, 

In [3]:
s3.download_file('dse-cohort5-group5', 'wildfire_capstone/gridMet.parquet.gz', 'gridMet.parquet.gz')

In [137]:
df = pd.read_parquet('gridMet.parquet.gz')

In [138]:
print(df.shape)
df = df[~df['precipitation_amount_mm'].isna()]
df = df.reset_index()
df = df.set_index('date')
print(df.shape)

(10676640, 14)
(6826300, 16)


In [139]:
cols = [c for c in df][2:]
df.head()

,latitude,longitude,precipitation_amount_mm,relative_humidity_%,specific_humidity_kg/kg,surface_downwelling_shortwave_flux_in_air_W m-2,wind_from_direction_Degrees Clockwise from north,wind_speed_m/s,max_air_temperature_K,min_air_temperature_K,burning_index_g_Unitless,dead_fuel_moisture_100hr_Percent,dead_fuel_moisture_1000hr_Percent,energy_release_component-g_Unitless,potential_evapotranspiration_mm,mean_vapor_pressure_deficit_kPa
date,,,,,,,,,,,,,,,,
1999-01-01,33.566667,-117.975000,0.0,40.3,0.00589,138.0,123.0,1.6,293.1,281.1,24.0,16.0,15.5,34.0,1.7,0.74
1999-01-01,33.566667,-117.933333,0.0,39.8,0.00590,137.2,123.0,1.6,293.1,281.2,24.0,16.0,15.5,35.0,1.7,0.74
1999-01-01,33.566667,-117.891667,0.0,38.2,0.00580,137.2,123.0,1.7,293.2,281.2,26.0,15.2,14.9,37.0,1.7,0.77
1999-01-01,33.566667,-117.850000,0.0,36.4,0.00567,137.3,49.0,1.8,293.3,280.3,27.0,15.0,14.8,38.0,1.8,0.76
1999-01-01,33.566667,-117.808333,0.0,33.8,0.00538,137.3,49.0,1.9,293.2,279.9,29.0,13.3,13.6,43.0,1.9,0.80


In [140]:
cols

['precipitation_amount_mm',
 'relative_humidity_%',
 'specific_humidity_kg/kg',
 'surface_downwelling_shortwave_flux_in_air_W m-2',
 'wind_from_direction_Degrees Clockwise from north',
 'wind_speed_m/s',
 'max_air_temperature_K',
 'min_air_temperature_K',
 'burning_index_g_Unitless',
 'dead_fuel_moisture_100hr_Percent',
 'dead_fuel_moisture_1000hr_Percent',
 'energy_release_component-g_Unitless',
 'potential_evapotranspiration_mm',
 'mean_vapor_pressure_deficit_kPa']

In [141]:
windows = ['7d', '14d', '30d']
windows

['7d', '14d', '30d']

In [142]:
result = df.reset_index().set_index(['date', 'latitude', 'longitude'])
print(result.shape)
for window in tqdm(windows):
    name_map = {n: "{}_mean_{}".format(window, n) for n in cols}
    tmp = df
    tmp = tmp.groupby(['latitude', 'longitude'])
    tmp = tmp.rolling(window).mean(skipna=True)
    tmp = tmp.rename(columns=name_map)
    result = result.join(
        tmp.drop(['latitude', 'longitude'], axis=1).reset_index()\
        .set_index(['date', 'latitude', 'longitude'])
    )
    print(result.shape)

(6826300, 14)


C:\Users\Mike\AppData\Local\Continuum\anaconda3\envs\geo\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


(6826300, 28)
(6826300, 42)
(6826300, 56)



In [143]:
print(result.shape)
for window in tqdm(windows):
    name_map = {n: "{}_max_{}".format(window, n) for n in cols}
    tmp = df
    tmp = tmp.groupby(['latitude', 'longitude'])
    tmp = tmp.rolling(window).max(skipna=True)
    tmp = tmp.rename(columns=name_map)
    result = result.join(
        tmp.drop(['latitude', 'longitude'], axis=1).reset_index()\
        .set_index(['date', 'latitude', 'longitude'])
    )
    print(result.shape)

(6826300, 56)


C:\Users\Mike\AppData\Local\Continuum\anaconda3\envs\geo\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


(6826300, 70)
(6826300, 84)
(6826300, 98)



In [144]:
result.head(10)

precipitation_amount_mm  \
date       latitude  longitude                              
1999-01-01 33.566667 -117.975000                      0.0   
                     -117.933333                      0.0   
                     -117.891667                      0.0   
                     -117.850000                      0.0   
                     -117.808333                      0.0   
                     -117.766667                      0.0   
                     -117.725000                      0.0   
                     -117.683333                      0.0   
                     -117.641667                      0.0   
                     -117.600000                      0.0   

                                  relative_humidity_%  \
date       latitude  longitude                          
1999-01-01 33.566667 -117.975000                 40.3   
                     -117.933333                 39.8   
                     -117.891667                 38.2   
                     -117.850000                 36.4   
                     -117.808333                 33.8   
                     -117.766667                 31.5   
                     -117.725000                 31.8   
                     -117.683333                 32.3   
                     -117.641667                 30.4   
                     -117.600000                 28.3   

                                  specific_humidity_kg/kg  \
date       latitude  longitude                              
1999-01-01 33.566667 -117.975000                  0.00589   
                     -117.933333                  0.00590   
                     -117.891667                  0.00580   
                     -117.850000                  0.00567   
                     -117.808333                  0.00538   
                     -117.766667                  0.00495   
                     -117.725000                  0.00504   
                     -117.683333                  0.00519   
                     -117.641667                  0.00504   
                     -117.600000                  0.00482   

                                  surface_downwelling_shortwave_flux_in_air_W m-2  \
date       latitude  longitude                                                      
1999-01-01 33.566667 -117.975000                                            138.0   
                     -117.933333                                            137.2   
                     -117.891667                                            137.2   
                     -117.850000                                            137.3   
                     -117.808333                                            137.3   
                     -117.766667                                            137.4   
                     -117.725000                                            137.4   
                     -117.683333                                            137.5   
                     -117.641667                                            137.0   
                     -117.600000                                            136.4   

                                  wind_from_direction_Degrees Clockwise from north  \
date       latitude  longitude                                                       
1999-01-01 33.566667 -117.975000                                             123.0   
                     -117.933333                                             123.0   
                     -117.891667                                             123.0   
                     -117.850000                                              49.0   
                     -117.808333                                              49.0   
                     -117.766667                                              49.0   
                     -117.725000                                              36.0   
                     -117.683333                                              36.0   
  

In [146]:
[col for col in result]

['precipitation_amount_mm',
 'relative_humidity_%',
 'specific_humidity_kg/kg',
 'surface_downwelling_shortwave_flux_in_air_W m-2',
 'wind_from_direction_Degrees Clockwise from north',
 'wind_speed_m/s',
 'max_air_temperature_K',
 'min_air_temperature_K',
 'burning_index_g_Unitless',
 'dead_fuel_moisture_100hr_Percent',
 'dead_fuel_moisture_1000hr_Percent',
 'energy_release_component-g_Unitless',
 'potential_evapotranspiration_mm',
 'mean_vapor_pressure_deficit_kPa',
 '7d_mean_precipitation_amount_mm',
 '7d_mean_relative_humidity_%',
 '7d_mean_specific_humidity_kg/kg',
 '7d_mean_surface_downwelling_shortwave_flux_in_air_W m-2',
 '7d_mean_wind_from_direction_Degrees Clockwise from north',
 '7d_mean_wind_speed_m/s',
 '7d_mean_max_air_temperature_K',
 '7d_mean_min_air_temperature_K',
 '7d_mean_burning_index_g_Unitless',
 '7d_mean_dead_fuel_moisture_100hr_Percent',
 '7d_mean_dead_fuel_moisture_1000hr_Percent',
 '7d_mean_energy_release_component-g_Unitless',
 '7d_mean_potential_evapotranspi

In [145]:
result.to_parquet('timeLagged.parquet.gzip', compression='gzip')